In [26]:
import json

from web3 import Web3
import pandas as pd

from datetime import datetime

import matplotlib.pyplot as plt
import bisect

# Import python dotenv
from dotenv import load_dotenv
import numpy as np
import os

from operator import itemgetter

from tqdm import tqdm, trange

# Import sqlalchemy
from sqlalchemy import create_engine

from sqlalchemy import MetaData, Table, String, Column, Text, DateTime, Boolean, Integer, BigInteger, Float, ForeignKey, Numeric
from sqlalchemy.orm.mapper import Mapper
from datetime import datetime

from pymongo import MongoClient, UpdateOne, DESCENDING, ASCENDING, InsertOne, DeleteOne


load_dotenv(override=True)

True

In [2]:
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))


In [3]:
postgres_uri = os.getenv("POSTGRESQL_URI")

assert postgres_uri is not None, "POSTGRESQL_URI is not set in .env file"

engine = create_engine(postgres_uri)

In [27]:
# Connect to mongodb
client = MongoClient(os.getenv("MONGO_URI"))

# Get database
mempool = client.transactions.mempool

mempool.estimated_document_count()

394313

In [18]:
metadata = MetaData()

## Define Swap schema

In [19]:
# Swap table:

"""
block_timestamp     TIMESTAMP	NULLABLE                        
block_number        INTEGER	    NULLABLE                        
transaction_hash    STRING	    NULLABLE                            
log_index           INTEGER	    NULLABLE                    
sender              STRING	    NULLABLE                
recipient           STRING	    NULLABLE                    
amount0             STRING	    NULLABLE                
amount1             STRING	    NULLABLE                
sqrtPriceX96        STRING	    NULLABLE                        
liquidity           STRING	    NULLABLE                    
tick                STRING	    NULLABLE                
address             STRING	    NULLABLE                
from_address        STRING	    NULLABLE                        
to_address          STRING	    NULLABLE                    
transaction_index   INTEGER	    NULLABLE                               
"""

'\nblock_timestamp     TIMESTAMP\tNULLABLE                        \nblock_number        INTEGER\t    NULLABLE                        \ntransaction_hash    STRING\t    NULLABLE                            \nlog_index           INTEGER\t    NULLABLE                    \nsender              STRING\t    NULLABLE                \nrecipient           STRING\t    NULLABLE                    \namount0             STRING\t    NULLABLE                \namount1             STRING\t    NULLABLE                \nsqrtPriceX96        STRING\t    NULLABLE                        \nliquidity           STRING\t    NULLABLE                    \ntick                STRING\t    NULLABLE                \naddress             STRING\t    NULLABLE                \nfrom_address        STRING\t    NULLABLE                        \nto_address          STRING\t    NULLABLE                    \ntransaction_index   INTEGER\t    NULLABLE                               \n'

In [63]:
from sqlalchemy import Column, Integer, String, DateTime, Boolean
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Swap(Base):
    __tablename__ = "swaps"

    # id = Column(Integer, primary_key=True, index=True, autoincrement=True)
    transaction_hash = Column(String, index=True)
    block_timestamp = Column(DateTime, nullable=False)
    block_number = Column(Integer, nullable=False, index=True, primary_key=True)
    transaction_index = Column(Integer, nullable=False, primary_key=True)
    log_index = Column(Integer, nullable=False, primary_key=True)
    sender = Column(String, nullable=False, index=True)
    recipient = Column(String, nullable=False, index=True)
    amount0 = Column(String, nullable=False)
    amount1 = Column(String, nullable=False)
    sqrtPriceX96 = Column(String, nullable=False)
    liquidity = Column(String, nullable=False)
    tick = Column(String, nullable=False)
    address = Column(String, nullable=False, index=True)
    from_address = Column(String, nullable=False, index=True)
    to_address = Column(String, nullable=False, index=True)
    from_mempool = Column(Boolean)

In [64]:
# Create a Session class bound to this engine
Session = sessionmaker(bind=engine)

# Now create the table
Base.metadata.create_all(engine)

## Ingest data from the blockchain into the Swap schema

In [66]:
# Load in the ABIs
with open('abi/UniswapV3Pool.json', 'r') as f:
    uniswap_v3_pool_abi = json.load(f)

with open('abi/UniswapV2Pair.json', 'r') as f:
    uniswap_v2_pair_abi = json.load(f)

def v3_swaps(tx_hash):
    # Get transaction receipt
    tx_receipt = w3.eth.get_transaction_receipt(tx_hash)

    liquidity_events = []

    for log in tx_receipt['logs']:

        contract = w3.eth.contract(abi=uniswap_v3_pool_abi, address=log['address'])
    
        # Parse the logs for Swap, Mint, and Burn events
        try:
            event_data = contract.events.Swap().process_log(log)
        except:
            continue

        liquidity_events.append(event_data)

    return liquidity_events

In [67]:
from datetime import datetime
from pydantic import BaseModel

class SwapArgs(BaseModel):
    sender: str
    recipient: str
    amount0: int
    amount1: int
    sqrtPriceX96: int
    liquidity: int
    tick: int

class SwapData(BaseModel):
    args: SwapArgs
    blockNumber: int
    event: str
    logIndex: int
    transactionIndex: int
    address: str
    blockNumber: int

In [99]:
start_block = 17_495_444
blocks_to_process = 1_000

swaps_to_insert = []

it = trange(start_block, start_block + blocks_to_process)
for block_number in it:
    block = w3.eth.get_block(block_number)
    
    if 'transactions' not in block:
        with open(f'./errors.txt', 'a') as f:
            f.writelines(f'Block {block_number} has no transactions\n')
        continue

    block_timestamp = datetime.fromtimestamp(block['timestamp'] if 'timestamp' in block else 0)

    for transaction in block['transactions']:

        # Get transaction hash
        tx_hash = w3.to_hex(transaction) # type: ignore

        # Get swaps from transaction
        swaps = v3_swaps(tx_hash)

        for swap in swaps:

            swap = SwapData(**swap)

            # Check if this transaction is in the mempool database from mongo
            from_mempool = bool(mempool.find_one({'hash': tx_hash}))

            swap_to_insert = Swap(
                transaction_hash=tx_hash,
                block_timestamp=block_timestamp,
                block_number=block_number,
                log_index=swap.logIndex,
                sender=swap.args.sender,
                recipient=swap.args.recipient,
                amount0=str(swap.args.amount0),
                amount1=str(swap.args.amount1),
                sqrtPriceX96=str(swap.args.sqrtPriceX96),
                liquidity=str(swap.args.liquidity),
                tick=str(swap.args.tick),
                address=swap.address,
                from_address=swap.args.sender,
                to_address=swap.args.recipient,
                transaction_index=swap.transactionIndex,
                from_mempool=from_mempool,
            )

            swaps_to_insert.append(swap_to_insert)
            it.set_postfix({"swaps_to_insert": len(swaps_to_insert)})


    # Checkpoint if we get more than 100 swaps
    if len(swaps_to_insert) > 100:
        # Insert the swaps into the database
        with Session() as session:
            for swap in swaps_to_insert:
                session.merge(swap)
            session.commit()

        swaps_to_insert = []


 24%|██▍       | 244/1000 [10:57<33:57,  2.69s/it, swaps_to_insert=26] 


KeyboardInterrupt: 

In [98]:
a = [x['hash'] for x in mempool.find(sort=[('ts', ASCENDING)]).skip(20).limit(10)]

# all(a == a.lower() for a in a)

a

['0xf17dabeadc1c5968387650ebdaab0b14b97168a7dbc096b652503ddd9aae42e4',
 '0x8ebe4d9171bd97bc026400de168aff6f0e78b23d1350f0e347a6cd5884ec2e42',
 '0x8b6f95d7c004a0996559df3f1c87e22698909c46bb429cc6ee8f35884d458c57',
 '0xca477dee6bef4733e03c6e45ffa3777e124acac488464dfe0db16b7dece24188',
 '0x528f0742665e6ff4b442a1c8c569070449fa256b86bd15f84f97aae258ee7d0d',
 '0x12916e6fb5c2448862bcf7276ac40fbd28ee35e7f29ab95ae57e796e5fd93c7e',
 '0x64291b6c47130673ff1af5020cae0128671e70449bd15a84bdb501b896ea80e5',
 '0x487d8033169bcacb8c4c9541f5c66126aa89fd4493984c051bc237c70de10ac3',
 '0x95edf1efe3014d47a31a0c9ab217122fe9be60527c2e4474f0f5a5ceb312c260',
 '0x20386bb814423cbcf8c2f707427a55e62b397896212a4d8f6ba2c85ea6817bba']

In [84]:
'0x15db41e167afa3daefff6bec3b2198ceda006041029b41b457115e6bac0cd345' in a

True

In [86]:
bool(mempool.find_one({'hash': '0x15db41e167afa3daefff6bec3b2198ceda006041029b41b457115e6bac0cd345'}))

True